# 用 pandas 分析CSV資料

In [1]:
import numpy as np
import pandas as pd

# 原先完整的資料
# df = pd.read_excel("online-retail.xlsx")

# 去除退貨資料
df = pd.read_csv('data/afterclean-without-refund.csv')


### (耗時，先拿掉) 顯示 Profile Report


In [2]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile.to_widgets()

## profile.to_notebook_iframe()

### 做一些準備手續
- 將InvoiceDate 的 dtype轉成 datetime
- 排除 StockCode 字串長度小於5的資料。(例如)

In [3]:
# 原本 InvoiceDate 的 dtype 是 object，轉成 datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Show All Columns
df.info()

# 一些 filter 條件
# StockCode : 排除字串長度小於5，例如：M,D,POST...
# PS. 初期是直接讀取原資料，所以會用到
stockcode_mask = df['StockCode'].str.len() >= 5

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32960 entries, 0 to 32959
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   32960 non-null  int64         
 1   InvoiceNo    32960 non-null  int64         
 2   StockCode    32960 non-null  object        
 3   Description  32960 non-null  object        
 4   Quantity     32960 non-null  int64         
 5   InvoiceDate  32960 non-null  datetime64[ns]
 6   UnitPrice    32960 non-null  float64       
 7   CustomerID   32960 non-null  int64         
 8   Country      32960 non-null  object        
 9   TotalPrice   32960 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 2.5+ MB


### 區分國家 列出 銷售總金額(不含運費) 的 中位數、最小值、最大值

In [4]:
country_group = df.loc[stockcode_mask].groupby('Country')
country_group.agg({'TotalPrice': ['median', 'min', 'max', 'sum']})

TotalPrice                           
                         median    min      max        sum
Country                                                   
Australia                30.000   2.08   270.00    4082.84
Austria                  19.500  15.00   142.80     865.95
Bahrain                  25.500  19.80    25.50      96.30
Belgium                  15.000   2.50    87.04    2015.36
Brazil                   17.850  17.85    20.40      56.10
Canada                   17.100   2.50    30.00     385.78
Channel Islands          20.400  10.20   358.00    1720.75
Cyprus                   18.725   1.26   196.35    1427.71
Czech Republic           39.600  35.76    46.80     122.16
Denmark                  20.000   9.36   166.00    1166.47
EIRE                     16.600   1.90   629.10   14151.97
European Community       42.500  23.40    51.00     159.40
Finland                  19.900   2.34   270.00    6290.92
France                   16.600   0.42   475.20    9699.11
Germany                  16.350   1.25   297.00   14267.70
Greece                   15.600   9.90   135.84     225.24
Iceland                  21.450  10.20   106.20     656.55
Israel                   10.500   1.26    47.40     403.98
Italy                    17.000   6.72    81.60    1113.02
Japan                   100.800  10.20   187.50    1441.20
Lebanon                  15.150  12.60    17.70      30.30
Lithuania                39.600  39.60    71.40     150.60
Malta                    17.000   2.28    30.00     302.74
Netherlands             113.760   4.20   652.80   14559.16
Norway                   19.250   5.80   270.00    3559.41
Poland                   15.000   8.40    40.68     413.54
Portugal                 16.600   0.84    53.55    1485.72
RSA                      16.350  15.60    20.80      52.75
Singapore                16.600   2.34    81.60     402.82
Spain                    14.925   0.84  1350.00   12678.92
Sweden                   16.250   6.96  1188.00    1711.28
Switzerland              17.850   5.04   262.80    4261.22
USA                      20.600  10.50    33.20     266.50
United Arab Emirates     65.400  65.40    65.40      65.40
United Kingdom            9.300   0.00  4921.50  630088.98
Unspecified               2.100   0.19    27.90     120.59

### 打算採用

#### 顯示每個月的銷售金額
- 排除一些 D,M,POST...

In [6]:
# 計算送銷售金額
df.loc[stockcode_mask]['TotalPrice'].sum(axis=0)

730498.4400000001

In [7]:
# 顯示每個月的銷售金額  (排除一些 D,M,POST...)
df.loc[stockcode_mask].groupby(df.InvoiceDate.dt.to_period("M"))['TotalPrice'].sum()

# TODO: 如果在 iOS上不知道如何處理，就在這裡輸出一個 CSV 檔

InvoiceDate
2010-12    62116.47
2011-01    65539.88
2011-02    46343.55
2011-03    57780.45
2011-04    49653.68
2011-05    59704.99
2011-06    50246.36
2011-07    58357.50
2011-08    54801.66
2011-09    60596.34
2011-10    63023.90
2011-11    83577.77
2011-12    18755.89
Freq: M, Name: TotalPrice, dtype: float64

#### StockCode 商品銷售數量 Top 10

In [8]:
# 取出 Stock 的銷售數量表
stock_sold_qty = df.groupby(['StockCode', 'Description'])['Quantity'].sum()     # 加總數量 方法1
# stock_sold_qty = df.groupby(['StockCode','Description']).agg({'Quantity':sum})  # 加總數量 方法2
# stock_sold_qty.to_csv('商品銷售數量.csv') # 輸出檔案檢查一下

# 銷售數量 Top 10
stock_sold_qty.nlargest(10)

StockCode  Description                       
85099B     JUMBO BAG RED RETROSPOT               46181
85123A     WHITE HANGING HEART T-LIGHT HOLDER    36725
85099F     JUMBO BAG STRAWBERRY                  16807
85099C     JUMBO  BAG BAROQUE BLACK WHITE        12793
84970S     HANGING HEART ZINC T-LIGHT HOLDER      8470
16161P     WRAP ENGLISH ROSE                      7226
75049L     LARGE CIRCULAR MIRROR MOBILE           6821
82494L     WOODEN FRAME ANTIQUE WHITE             6282
16156S     WRAP PINK FAIRY CAKES                  5800
84970L     SINGLE HEART ZINC T-LIGHT HOLDER       5700
Name: Quantity, dtype: int64

### StockCode 商品銷售金額 Top 10

In [9]:
# 銷售金額 Top 10  (排除一些 D,M,POST...)
df.loc[stockcode_mask].groupby(['StockCode', 'Description'])['TotalPrice'].sum().nlargest(10)

StockCode  Description                       
85123A     WHITE HANGING HEART T-LIGHT HOLDER    100448.15
85099B     JUMBO BAG RED RETROSPOT                85220.78
85099F     JUMBO BAG STRAWBERRY                   30644.20
85099C     JUMBO  BAG BAROQUE BLACK WHITE         23675.04
15056N     EDWARDIAN PARASOL NATURAL              20696.71
82494L     WOODEN FRAME ANTIQUE WHITE             17625.07
84997D     CHILDRENS CUTLERY POLKADOT PINK        13796.60
84029E     RED WOOLLY HOTTIE WHITE HEART.         13430.51
84078A     SET/4 WHITE RETRO STORAGE CUBES        13007.15
47590B     PINK HAPPY BIRTHDAY BUNTING            11688.15
Name: TotalPrice, dtype: float64

#### Customer 顧客花錢金額 Top 10

In [15]:
# Customer 銷售金額 Top 10, 正常來說 Customer的Country 不會亂變 (排除一些 D,M,POST...)
# df.loc[stockcode_mask].groupby(['CustomerID', 'Country'])['TotalPrice'].sum().nlargest(10)

# Customer 銷售金額 Top 10, 正常來說 Customer的Country 不會亂變 (沒排除)
df.groupby(['CustomerID', 'Country'])['TotalPrice'].sum().nlargest(10)

CustomerID  Country       
15769       United Kingdom    21681.62
15749       United Kingdom    15598.50
14646       Netherlands       14031.10
17450       United Kingdom    13703.76
13694       United Kingdom    12904.28
13777       United Kingdom    10794.97
14911       EIRE              10463.62
13798       United Kingdom    10167.56
17511       United Kingdom    10109.99
15838       United Kingdom     9774.30
Name: TotalPrice, dtype: float64

#### Country 哪個國家最會花錢 Top 10

In [14]:
# 區分 Country 的銷售金額 Top 10 (排除一些 D,M,POST...)
# df.loc[stockcode_mask].groupby('Country')['TotalPrice'].sum().nlargest(10)

# 區分 Country 的銷售金額 Top 10 (沒排除)
df.groupby('Country')['TotalPrice'].sum().nlargest(10)

Country
United Kingdom    630088.98
Netherlands        14559.16
Germany            14267.70
EIRE               14151.97
Spain              12678.92
France              9699.11
Finland             6290.92
Switzerland         4261.22
Australia           4082.84
Norway              3559.41
Name: TotalPrice, dtype: float64

### 以國別之分，列出每筆運費 (後來去除掉運費資料，所以看不到資料)

In [12]:
# 區分國家 列出運費的 中位數、最小值、最大值
stock_group = df.loc[df['StockCode'] == 'POST'].groupby('Country')
stock_group.agg({'UnitPrice': ['median', 'min', 'max']})

Empty DataFrame
Columns: [(UnitPrice, median), (UnitPrice, min), (UnitPrice, max)]
Index: []

### 一些嘗試過的程式碼

In [ ]:
# df.groupby(pd.Grouper(freq='M'))
# df.groupby(df.InvoiceDate.dt.month)['Quantity'].sum() # 這個方法會把 2010-12及2011-12 月的混在一起

# 統計 Country 欄位的數值，各出現多少次
# country_count = df.value_counts('Country')
# print(country_count)
# df.loc[df['StockCode'] == 'POST']
# ----< 在試著列出 商品不同價格 >----
# stock_group = df.groupby(['StockCode', 'Description', 'UnitPrice'])
# stock_group.size()
# stock_group.groups
# -------------------------------
# stock_group.get_group("POST")
# 可知道 同個 Stock 是否有不同 Price
# print(stock_group['UnitPrice'].aggregate(['min', 'max']))

# list(stock_group)

# df.loc[df['Quantity'] == 0]  # 只是檢查有沒有數量為0的 銷售/退貨 項目
# df.loc[df['Quantity'] <= 0]  ＃ 退貨
# df.loc[df['UnitPrice'] <= 0]

# df.loc[df['CustomerID'] == 'nan']                 # 轉換後，空值會變成 nan
# df = df[df.line_race != 0]